In [9]:
using Rocket
using Distributions
using ReactiveMP
using BenchmarkTools

import Base: show

In [10]:
function createSubgraph(model, index::Int)
    noise_add = add!(model, AdditionNode())
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    yn    = add!(model, datavar(:yn, Float64))
    
    connect!(noise_add, :in2, noise)
    connect!(noise_add, :out, yn)
    
    return (yn, noise_add)
end

function createGraph(size::Int)    
    model = Model(DefaultMessageGate())
    
    c0_add = add!(model, AdditionNode())
    
    c0       = add!(model, constvar(:c0, 1.0))
    x0_prior = add!(model, datavar(:x0_prior, Normal{Float64}))
    
    connect!(c0_add, :in1, x0_prior)
    connect!(c0_add, :in2, c0)
    
    index = 1
    
    prev_c_add = c0_add
    
    xs = Vector{RandomVariable}(undef, size)
    ys = Vector{DataVariable}(undef, size)
    
    while index < size
        yn, noise_add = createSubgraph(model, index)
        xn            = add!(model, randomvar(:xn))
        
        c_add = add!(model, AdditionNode())
        cn    = add!(model, constvar(:cn, 1.0))
        
        connect!(prev_c_add, :out, xn)
        connect!(noise_add, :in1, xn)
        connect!(c_add, :in1, xn)
        connect!(c_add, :in2, cn)
        
        activate!(model, prev_c_add)
        activate!(model, noise_add)
        
        xs[index] = xn
        ys[index] = yn
        
        prev_c_add = c_add
        
        index += 1
    end
    
    last_noise_add = add!(model, AdditionNode());
    
    x_last     = add!(model, randomvar(:x_last))
    y_last     = add!(model, datavar(:ylast, Float64))
    last_noise = add!(model, constvar(:last_noise, Normal(0.0, sqrt(200.0))))
    
    connect!(prev_c_add, :out, x_last)
    connect!(last_noise_add, :in1, x_last)
    connect!(last_noise_add, :in2, last_noise)
    connect!(last_noise_add, :out, y_last)
    
    activate!(model, prev_c_add)
    activate!(model, last_noise_add)
    
    xs[size] = x_last
    ys[size] = y_last
    
    return (xs, ys, x0_prior)
end

createGraph (generic function with 1 method)

In [11]:
function smoothing(data)
    N = length(data)
    
    xs, ys, x_prior = createGraph(N);
    
    marginals     = Vector{Normal{Float64}}(undef, N)
    subscriptions = Vector{Teardown}(undef, N)
    
    @inbounds for (index, x) in enumerate(xs)
        subscriptions[index] = subscribe!(getmarginal(x), (d) -> marginals[index] = getdata(d))
    end
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    for i in 1:N
       update!(ys[i], data[i])
    end
    
    foreach(unsubscribe!, subscriptions)
    
    return marginals
end

smoothing (generic function with 1 method)

In [12]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [13]:
@time smoothing(data); # Initial compilation

  4.106547 seconds (10.77 M allocations: 568.126 MiB, 6.86% gc time)


In [14]:
@time smoothing(data); # Subsequent runs

  0.046613 seconds (396.49 k allocations: 24.831 MiB, 37.03% gc time)


In [15]:
@btime smoothing($data) # Performance benchmark

  28.197 ms (396485 allocations: 24.83 MiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=0.5364614884152861, σ=0.5773406469256954)
 Normal{Float64}(μ=1.5364614884152858, σ=0.5773406469256954)
 Normal{Float64}(μ=2.536461488415286, σ=0.5773406469256954)
 Normal{Float64}(μ=3.536461488415285, σ=0.5773406469256953)
 Normal{Float64}(μ=4.536461488415286, σ=0.5773406469256954)
 Normal{Float64}(μ=5.536461488415285, σ=0.5773406469256954)
 Normal{Float64}(μ=6.536461488415285, σ=0.5773406469256954)
 Normal{Float64}(μ=7.536461488415283, σ=0.5773406469256954)
 Normal{Float64}(μ=8.536461488415286, σ=0.5773406469256954)
 Normal{Float64}(μ=9.536461488415284, σ=0.5773406469256953)
 Normal{Float64}(μ=10.536461488415283, σ=0.5773406469256954)
 Normal{Float64}(μ=11.536461488415283, σ=0.5773406469256953)
 Normal{Float64}(μ=12.536461488415283, σ=0.5773406469256954)
 ⋮
 Normal{Float64}(μ=588.5364614884163, σ=0.577340646925696)
 Normal{Float64}(μ=589.5364614884162, σ=0.577340646925696)
 Normal{Float64}(μ=590.5364614884163, σ=0.57734064692569